# Plateau Width Analysis

In [1]:
import sys
sys.path.append('..')

## Load Data

In [2]:
import wfdb

def read_record(path, sampto=None):
    record = wfdb.rdrecord(path, sampto=sampto)
    annotation = wfdb.rdann(path, 'atr', sampto=sampto)
    trigger = trigger_points(annotation)
    return record, trigger

def read_records(directory, names, sampto=None):
    records, triggers = [], []
    for name in names:
        path = "/".join([directory, name])
        record, trigger = read_record(path, sampto)
        records.append(record)
        triggers.append(trigger)
    return records, triggers

In [3]:
import numpy as np
from raccoon.utils.annotationutils import trigger_points

# Clean ECG Signals
signal_dir = "../data/mitdb"
names = np.array([
    "100", "101", "102", "103", "104", "105", "106", "107", "108", "109"
])
sampto = 10000

clean_records, triggers = read_records(signal_dir, names, sampto)
clean_records, triggers = np.array(clean_records), np.array(triggers)

# Noise template
noise_path = '../data/noises/em'
noise_template = wfdb.rdrecord(noise_path)

## Add Noise

In [4]:
from raccoon.utils.noiseutils import apply_noise_db

noisy_records = [
    apply_noise_db(clean_record, noise_template, 6)
    for clean_record in clean_records]

noisy_records = np.array(noisy_records)

## Create Detector

In [5]:
from raccoon.detectors import XiangDetector

xd = XiangDetector(
    name='XD', batch_size=32, window_size=40, detection_size=20,
    aux_ratio=5, epochs=3)

Using TensorFlow backend.


## Train and Test

In [ ]:
from contextlib import redirect_stdout
from io import StringIO
from sklearn.model_selection import LeaveOneOut
from raccoon.utils.evaluationutils import trigger_metrics
from raccoon.utils.triggerutils import signal_to_points

cv = LeaveOneOut()

min_plateau_widths = [0, 5, 10, 15, 20]

metrics = []

for step_id, (train_indexes, test_indexes) in enumerate(cv.split(names), start=1):
    print("Step {}/{}".format(step_id, cv.get_n_splits(names)))
    
    train_records = clean_records[train_indexes]
    train_triggers = triggers[train_indexes]
    test_record = noisy_records[test_indexes][0]
    test_trigger = triggers[test_indexes][0]
    
    xd.reset()
    
    with redirect_stdout(StringIO()):
        xd.train(train_records, train_triggers)
    
    trigger_signal = xd.trigger_signal(test_record)
    detected_triggers = [
        signal_to_points(trigger_signal, min_width=mpw)
        for mpw in min_plateau_widths]
    
    metrics.append([trigger_metrics(test_trigger, detected_trigger, 5)
                    for detected_trigger in detected_triggers])

Step 1/10
Step 2/10
Step 3/10
Step 4/10
Step 5/10
Step 6/10
Step 7/10


In [ ]:
metrics